# NCAA ML Competition - Women's 2018

## Notes

Notes about what's in the data files: https://www.kaggle.com/c/womens-machine-learning-competition-2018/data

Starter Kernel might help: https://www.kaggle.com/juliaelliott/basic-starter-kernel-ncaa-women-s-dataset

In [1]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Load regular season data

In [2]:
df = pd.read_csv('WStage2DataFiles/WRegularSeasonDetailedResults.csv') ## regular season exploration
df.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
46337,2018,132,3195,68,3239,58,H,0,24,58,...,14,9,13,14,18,8,19,5,5,19
46338,2018,132,3251,60,3421,42,H,0,21,56,...,11,12,15,8,22,4,10,3,1,14
46339,2018,132,3311,69,3372,65,N,0,24,47,...,26,12,14,16,20,15,16,9,2,17
46340,2018,132,3343,63,3335,34,N,0,21,54,...,23,6,9,7,24,7,9,4,4,19
46341,2018,132,3384,66,3352,56,H,0,19,46,...,22,5,10,13,25,11,18,9,0,24


## Load the league that each team belongs to

In [3]:
df_teams = pd.read_csv('./WStage2DataFiles/' + 'WTeams.csv')
df_leagues = pd.merge(left=df_teams, right=pd.read_csv('./WStage2DataFiles/' + 'WLeagues.csv'), how='left', on=['TeamName'])
df_winning_team_leagues = df_leagues
df_winning_team_leagues = df_winning_team_leagues.rename(columns={'TeamID': 'WTeamID', 'LeagueName': 'WLeagueName'})
df_winning_team_leagues = df_winning_team_leagues.drop('TeamName', 1)
df_winning_team_leagues.head()

,WTeamID,WLeagueName
0,3101,Southland
1,3102,MWC
2,3103,MAC
3,3104,SEC
4,3105,SWAC


## Load and bin league performance (to dampen the effect of small variations)

In [4]:
## Bin league performance into 10 bins
df_league_perf = pd.read_pickle('league_performance')
df_league_perf['Leaguebin'] = pd.cut(df_league_perf['PctWins mean'], 24, labels=False)

df_league_perf = df_league_perf[['LeagueName', 'Leaguebin']]
df_team_league_perf = pd.merge(left=df_leagues, right=df_league_perf, how='left', on=['LeagueName'])

df_team_league_perf = df_team_league_perf[['TeamID', 'Leaguebin']]
df_team_league_perf.head()



,TeamID,Leaguebin
0,3101,1.0
1,3102,7.0
2,3103,2.0
3,3104,23.0
4,3105,0.0


In [5]:
##df = df[df['Season'] > 2012] ## 
##df = df[df['DayNum'] > 30]

## Inject winning and losing teams' leaugues into df
df_merged = pd.merge(left=df, right=df_winning_team_leagues, how='left', on=['WTeamID'])

df_winning_team_leagues = df_winning_team_leagues.rename(columns={'WTeamID': 'LTeamID', 'WLeagueName': 'LLeagueName'})
df = pd.merge(left=df_merged, right=df_winning_team_leagues, how='left', on=['LTeamID'])
df.head()


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,WLeagueName,LLeagueName
0,2010,11,3103,63,3237,49,H,0,23,54,...,10,11,27,11,23,7,6,19,MAC,Horizon
1,2010,11,3104,73,3399,68,N,0,26,62,...,27,14,26,7,20,4,2,27,SEC,OVC
2,2010,11,3110,71,3224,59,A,0,29,62,...,23,17,23,8,15,6,0,15,Patriot,MEAC
3,2010,11,3111,63,3267,58,A,0,27,52,...,25,22,22,15,11,14,5,14,Sun Belt,C-USA
4,2010,11,3119,74,3447,70,H,1,30,74,...,21,21,32,12,14,4,2,14,Patriot,NEC


## Feature Engineering
-  Operationalize the notion of Point Opportunities Developed (POD) and Opportunity Convertion Rate (OCR)
-  Demean and rescale the performance of teams by league and season
-  Bin performance to dampen small variations


In [6]:
df['WPOD'] = df.WFGA3 * 3 + (df.WFGA - df.WFGA3) * 2 + df.WFTA * 1
df['LPOD'] = df.LFGA3 * 3 + (df.LFGA - df.LFGA3) * 2 + df.LFTA * 1

df['WOCR'] = 1.0 * df.WScore / df.WPOD
df['LOCR'] = 1.0 * df.LScore / df.LPOD


In [7]:
## Isolate field goal percentage by team regardless of who won a game
df['WFGP'] = df.WFGM / df.WFGA ## Field goal percentage for the winning team
df['LFGP'] = df.LFGM / df.LFGA ## Field goal percentage for the losing team

df_WTeamFGP_game = df[['Season','WLeagueName', 'WTeamID','WFGP']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WFGP':'FGP'})
df_LTeamFGP_game = df[['Season','LLeagueName', 'LTeamID','LFGP']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LFGP':'FGP'})
df_TeamFGP_game = pd.concat([df_WTeamFGP_game, df_LTeamFGP_game])

df_TeamFGP_season = df_TeamFGP_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'FGP':['mean']})
df_TeamFGP_season.columns = df_TeamFGP_season.columns.droplevel(1)

df_FGP_season = df_TeamFGP_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'FGP':['mean','std']})

df_FGP_season.columns = [' '.join(col).strip() for col in df_FGP_season.columns.values] ## Flatten column name levels

df_TeamFGP_season_merged = pd.merge(left=df_TeamFGP_season, right=df_FGP_season, how='left', on=['Season', 'LeagueName'])

## Normalize field goal by demeaning and standardizing
df_TeamFGP_season_merged['FGP_norm'] = ((df_TeamFGP_season_merged['FGP'] - df_TeamFGP_season_merged['FGP mean'])/df_TeamFGP_season_merged['FGP std'])

## Bin normalized field goal percentage
df_TeamFGP_season_merged['FGPbin'] = pd.cut(df_TeamFGP_season_merged.FGP_norm, 24, labels=False)
df_TeamFGP_season_merged.tail()

,Season,LeagueName,TeamID,FGP,FGP mean,FGP std,FGP_norm,FGPbin
3113,2018,WCC,3339,0.388120,0.412622,0.078129,-0.313611,7
3114,2018,WCC,3360,0.411764,0.412622,0.078129,-0.010987,10
3115,2018,WCC,3362,0.415736,0.412622,0.078129,0.039848,10
3116,2018,WCC,3365,0.349105,0.412622,0.078129,-0.812980,3
3117,2018,WCC,3388,0.483855,0.412622,0.078129,0.911732,18


In [8]:
## Isolate free throw percentage by team regardless of who won a game
df['WFTP'] = df.WFTM / df.WFTA ## Field goal percentage for the winning team
df['LFTP'] = df.LFTM / df.LFTA ## Field goal percentage for the losing team

df_WTeamFTP_game = df[['Season','WLeagueName', 'WTeamID','WFTP']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WFTP':'FTP'})
df_LTeamFTP_game = df[['Season','LLeagueName', 'LTeamID','LFTP']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LFTP':'FTP'})
df_TeamFTP_game = pd.concat([df_WTeamFTP_game, df_LTeamFTP_game])

df_TeamFTP_season = df_TeamFTP_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'FTP':['mean']})
df_TeamFTP_season.columns = df_TeamFTP_season.columns.droplevel(1)

df_FTP_season = df_TeamFTP_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'FTP':['mean','std']})

df_FTP_season.columns = [' '.join(col).strip() for col in df_FTP_season.columns.values] ## Flatten column name levels

df_TeamFTP_season_merged = pd.merge(left=df_TeamFTP_season, right=df_FTP_season, how='left', on=['Season', 'LeagueName'])

## Normalize free throw by demeaning and standardizing
df_TeamFTP_season_merged['FTP_norm'] = ((df_TeamFTP_season_merged['FTP'] - df_TeamFTP_season_merged['FTP mean'])/df_TeamFTP_season_merged['FTP std'])

## Bin normalized free throw percentage
df_TeamFTP_season_merged['FTPbin'] = pd.cut(df_TeamFTP_season_merged.FTP_norm, 24, labels=False)
df_TeamFTP_season_merged.head()

,Season,LeagueName,TeamID,FTP,FTP mean,FTP std,FTP_norm,FTPbin
0,2010,??,3147,0.662420,0.662420,0.134582,0.000000,13
1,2010,AAC,3153,0.682553,0.659702,0.131597,0.173643,15
2,2010,AAC,3163,0.717560,0.659702,0.131597,0.439659,18
3,2010,AAC,3187,0.693580,0.659702,0.131597,0.257432,16
4,2010,AAC,3222,0.646145,0.659702,0.131597,-0.103019,12


In [9]:
## Isolate FGA by team regardless of who won a game

df_WTeamFGA_game = df[['Season','WLeagueName', 'WTeamID','WFGA']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WFGA':'FGA'})
df_LTeamFGA_game = df[['Season','LLeagueName', 'LTeamID','LFGA']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LFGA':'FGA'})
df_TeamFGA_game = pd.concat([df_WTeamFGA_game, df_LTeamFGA_game])

df_TeamFGA_season = df_TeamFGA_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'FGA':['mean']})
df_TeamFGA_season.columns = df_TeamFGA_season.columns.droplevel(1)

df_FGA_season = df_TeamFGA_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'FGA':['mean','std']})

df_FGA_season.columns = [' '.join(col).strip() for col in df_FGA_season.columns.values] ## Flatten column name levels

df_TeamFGA_season_merged = pd.merge(left=df_TeamFGA_season, right=df_FGA_season, how='left', on=['Season', 'LeagueName'])

## Normalize FGA by demeaning and standardizing
df_TeamFGA_season_merged['FGA_norm'] = ((df_TeamFGA_season_merged['FGA'] - df_TeamFGA_season_merged['FGA mean'])/df_TeamFGA_season_merged['FGA std'])

## Bin normalized FGA
df_TeamFGA_season_merged['FGAbin'] = pd.cut(df_TeamFGA_season_merged.FGA_norm, 24, labels=False)
df_TeamFGA_season_merged.head()

,Season,LeagueName,TeamID,FGA,FGA mean,FGA std,FGA_norm,FGAbin
0,2010,??,3147,47.551724,47.551724,8.086966,0.000000,10
1,2010,AAC,3153,52.172414,58.855956,7.818871,-0.854796,4
2,2010,AAC,3163,60.812500,58.855956,7.818871,0.250234,11
3,2010,AAC,3187,55.354839,58.855956,7.818871,-0.447778,7
4,2010,AAC,3222,62.129032,58.855956,7.818871,0.418612,13


In [10]:
## Isolate FTP by team regardless of who won a game

df_WTeamFTA_game = df[['Season','WLeagueName', 'WTeamID','WFTA']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WFTA':'FTA'})
df_LTeamFTA_game = df[['Season','LLeagueName', 'LTeamID','LFTA']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LFTA':'FTA'})
df_TeamFTA_game = pd.concat([df_WTeamFTA_game, df_LTeamFTA_game])

df_TeamFTA_season = df_TeamFTA_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'FTA':['mean']})
df_TeamFTA_season.columns = df_TeamFTA_season.columns.droplevel(1)

df_FTA_season = df_TeamFTA_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'FTA':['mean','std']})

df_FTA_season.columns = [' '.join(col).strip() for col in df_FTA_season.columns.values] ## Flatten column name levels

df_TeamFTA_season_merged = pd.merge(left=df_TeamFTA_season, right=df_FTA_season, how='left', on=['Season', 'LeagueName'])

## Normalize FTP by demeaning and standardizing
df_TeamFTA_season_merged['FTA_norm'] = ((df_TeamFTA_season_merged['FTA'] - df_TeamFTA_season_merged['FTA mean'])/df_TeamFTA_season_merged['FTA std'])

## Bin normalized FTP
df_TeamFTA_season_merged['FTAbin'] = pd.cut(df_TeamFTA_season_merged.FTA_norm, 24, labels=False)
df_TeamFTA_season_merged.head()

,Season,LeagueName,TeamID,FTA,FTA mean,FTA std,FTA_norm,FTAbin
0,2010,??,3147,15.896552,15.896552,6.241250,0.000000,12
1,2010,AAC,3153,16.137931,19.531856,7.289239,-0.465608,7
2,2010,AAC,3163,18.281250,19.531856,7.289239,-0.171569,10
3,2010,AAC,3187,21.870968,19.531856,7.289239,0.320899,16
4,2010,AAC,3222,22.225806,19.531856,7.289239,0.369579,17


In [11]:
## Isolate assists by team regardless of who won a game

df_WTeamAst_game = df[['Season','WLeagueName', 'WTeamID','WAst']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WAst':'Ast'})
df_LTeamAst_game = df[['Season','LLeagueName', 'LTeamID','LAst']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LAst':'Ast'})
df_TeamAst_game = pd.concat([df_WTeamAst_game, df_LTeamAst_game])

df_TeamAst_season = df_TeamAst_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'Ast':['mean']})
df_TeamAst_season.columns = df_TeamAst_season.columns.droplevel(1)

df_Ast_season = df_TeamAst_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'Ast':['mean','std']})

df_Ast_season.columns = [' '.join(col).strip() for col in df_Ast_season.columns.values] ## Flatten column name levels

df_TeamAst_season_merged = pd.merge(left=df_TeamAst_season, right=df_Ast_season, how='left', on=['Season', 'LeagueName'])

## Normalize assists by demeaning and standardizing
df_TeamAst_season_merged['Ast_norm'] = ((df_TeamAst_season_merged['Ast'] - df_TeamAst_season_merged['Ast mean'])/df_TeamAst_season_merged['Ast std'])

## Bin normalized assists
df_TeamAst_season_merged['Astbin'] = pd.cut(df_TeamAst_season_merged.Ast_norm, 24, labels=False)
df_TeamAst_season_merged.head()

,Season,LeagueName,TeamID,Ast,Ast mean,Ast std,Ast_norm,Astbin
0,2010,??,3147,9.689655,9.689655,3.818222,0.000000,9
1,2010,AAC,3153,11.103448,13.681440,4.749376,-0.542807,5
2,2010,AAC,3163,19.468750,13.681440,4.749376,1.218541,18
3,2010,AAC,3187,16.258065,13.681440,4.749376,0.542518,13
4,2010,AAC,3222,11.935484,13.681440,4.749376,-0.367618,6


In [12]:
## Isolate blocks by team regardless of who won a game

df_WTeamBlk_game = df[['Season','WLeagueName', 'WTeamID','WBlk']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WBlk':'Blk'})
df_LTeamBlk_game = df[['Season','LLeagueName', 'LTeamID','LBlk']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LBlk':'Blk'})
df_TeamBlk_game = pd.concat([df_WTeamBlk_game, df_LTeamBlk_game])

df_TeamBlk_season = df_TeamBlk_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'Blk':['mean']})
df_TeamBlk_season.columns = df_TeamBlk_season.columns.droplevel(1)

df_Blk_season = df_TeamBlk_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'Blk':['mean','std']})

df_Blk_season.columns = [' '.join(col).strip() for col in df_Blk_season.columns.values] ## Flatten column name levels

df_TeamBlk_season_merged = pd.merge(left=df_TeamBlk_season, right=df_Blk_season, how='left', on=['Season', 'LeagueName'])

## Normalize blocks by demeaning and standardizing
df_TeamBlk_season_merged['Blk_norm'] = ((df_TeamBlk_season_merged['Blk'] - df_TeamBlk_season_merged['Blk mean'])/df_TeamBlk_season_merged['Blk std'])

## Bin normalized blocks
df_TeamBlk_season_merged['Blkbin'] = pd.cut(df_TeamBlk_season_merged.Blk_norm, 24, labels=False)
df_TeamBlk_season_merged.head()

,Season,LeagueName,TeamID,Blk,Blk mean,Blk std,Blk_norm,Blkbin
0,2010,??,3147,3.275862,3.275862,1.980198,0.000000,9
1,2010,AAC,3153,2.241379,3.318560,2.348975,-0.458575,5
2,2010,AAC,3163,5.500000,3.318560,2.348975,0.928678,17
3,2010,AAC,3187,2.483871,3.318560,2.348975,-0.355342,6
4,2010,AAC,3222,3.451613,3.318560,2.348975,0.056643,10


In [13]:
## Isolate offensive rebounds by team regardless of who won a game

df_WTeamOR_game = df[['Season','WLeagueName', 'WTeamID','WOR']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WOR':'OR'})
df_LTeamOR_game = df[['Season','LLeagueName', 'LTeamID','LOR']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LOR':'OR'})
df_TeamOR_game = pd.concat([df_WTeamOR_game, df_LTeamOR_game])

df_TeamOR_season = df_TeamOR_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'OR':['mean']})
df_TeamOR_season.columns = df_TeamOR_season.columns.droplevel(1)

df_OR_season = df_TeamOR_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'OR':['mean','std']})

df_OR_season.columns = [' '.join(col).strip() for col in df_OR_season.columns.values] ## Flatten column name levels

df_TeamOR_season_merged = pd.merge(left=df_TeamOR_season, right=df_OR_season, how='left', on=['Season', 'LeagueName'])

## Normalize offensive rebounds by demeaning and standardizing
df_TeamOR_season_merged['OR_norm'] = ((df_TeamOR_season_merged['OR'] - df_TeamOR_season_merged['OR mean'])/df_TeamOR_season_merged['OR std'])

## Bin normalized offensive rebounds
df_TeamOR_season_merged['ORbin'] = pd.cut(df_TeamOR_season_merged.OR_norm, 24, labels=False)
df_TeamOR_season_merged.head()

,Season,LeagueName,TeamID,OR,OR mean,OR std,OR_norm,ORbin
0,2010,??,3147,10.965517,10.965517,6.304662,0.000000,11
1,2010,AAC,3153,10.448276,14.132964,4.592753,-0.802283,5
2,2010,AAC,3163,13.125000,14.132964,4.592753,-0.219468,10
3,2010,AAC,3187,12.516129,14.132964,4.592753,-0.352040,9
4,2010,AAC,3222,14.451613,14.132964,4.592753,0.069381,12


In [14]:
## Isolate defensive rebounds by team regardless of who won a game

df_WTeamDR_game = df[['Season','WLeagueName', 'WTeamID','WDR']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WDR':'DR'})
df_LTeamDR_game = df[['Season','LLeagueName', 'LTeamID','LDR']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LDR':'DR'})
df_TeamDR_game = pd.concat([df_WTeamDR_game, df_LTeamDR_game])

df_TeamDR_season = df_TeamDR_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'DR':['mean']})
df_TeamDR_season.columns = df_TeamDR_season.columns.droplevel(1)

df_DR_season = df_TeamDR_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'DR':['mean','std']})

df_DR_season.columns = [' '.join(col).strip() for col in df_DR_season.columns.values] ## Flatten column name levels

df_TeamDR_season_merged = pd.merge(left=df_TeamDR_season, right=df_DR_season, how='left', on=['Season', 'LeagueName'])

## Normalize defensive rebounds by demeaning and standardizing
df_TeamDR_season_merged['DR_norm'] = ((df_TeamDR_season_merged['DR'] - df_TeamDR_season_merged['DR mean'])/df_TeamDR_season_merged['DR std'])

## Bin normalized defensive rebounds
df_TeamDR_season_merged['DRbin'] = pd.cut(df_TeamDR_season_merged.DR_norm, 24, labels=False)
df_TeamDR_season_merged.head()


,Season,LeagueName,TeamID,DR,DR mean,DR std,DR_norm,DRbin
0,2010,??,3147,20.724138,20.724138,4.534759,0.000000,12
1,2010,AAC,3153,23.793103,25.252078,5.559791,-0.262415,9
2,2010,AAC,3163,30.625000,25.252078,5.559791,0.966389,22
3,2010,AAC,3187,27.129032,25.252078,5.559791,0.337594,15
4,2010,AAC,3222,26.548387,25.252078,5.559791,0.233158,14


In [15]:
## Isolate steals by team regardless of who won a game

df_WTeamStl_game = df[['Season','WLeagueName', 'WTeamID','WStl']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WStl':'Stl'})
df_LTeamStl_game = df[['Season','LLeagueName', 'LTeamID','LStl']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LStl':'Stl'})
df_TeamStl_game = pd.concat([df_WTeamStl_game, df_LTeamStl_game])

df_TeamStl_season = df_TeamStl_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'Stl':['mean']})
df_TeamStl_season.columns = df_TeamStl_season.columns.droplevel(1)

df_Stl_season = df_TeamStl_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'Stl':['mean','std']})

df_Stl_season.columns = [' '.join(col).strip() for col in df_Stl_season.columns.values] ## Flatten column name levels

df_TeamStl_season_merged = pd.merge(left=df_TeamStl_season, right=df_Stl_season, how='left', on=['Season', 'LeagueName'])

## Normalize steals by demeaning and standardizing
df_TeamStl_season_merged['Stl_norm'] = ((df_TeamStl_season_merged['Stl'] - df_TeamStl_season_merged['Stl mean'])/df_TeamStl_season_merged['Stl std'])

## Bin normalized steals
df_TeamStl_season_merged['Stlbin'] = pd.cut(df_TeamStl_season_merged.Stl_norm, 24, labels=False)
df_TeamStl_season_merged.head()

,Season,LeagueName,TeamID,Stl,Stl mean,Stl std,Stl_norm,Stlbin
0,2010,??,3147,6.482759,6.482759,3.031038,0.000000,8
1,2010,AAC,3153,6.103448,9.066482,3.675143,-0.806236,2
2,2010,AAC,3163,9.375000,9.066482,3.675143,0.083947,8
3,2010,AAC,3187,9.032258,9.066482,3.675143,-0.009312,8
4,2010,AAC,3222,8.677419,9.066482,3.675143,-0.105863,7


In [16]:
## Isolate POD by team regardless of who won a game

df_WTeamPOD_game = df[['Season','WLeagueName', 'WTeamID','WPOD']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WPOD':'POD'})
df_LTeamPOD_game = df[['Season','LLeagueName', 'LTeamID','LPOD']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LPOD':'POD'})
df_TeamPOD_game = pd.concat([df_WTeamPOD_game, df_LTeamPOD_game])

df_TeamPOD_season = df_TeamPOD_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'POD':['mean']})
df_TeamPOD_season.columns = df_TeamPOD_season.columns.droplevel(1)

df_POD_season = df_TeamPOD_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'POD':['mean','std']})

df_POD_season.columns = [' '.join(col).strip() for col in df_POD_season.columns.values] ## Flatten column name levels

df_TeamPOD_season_merged = pd.merge(left=df_TeamPOD_season, right=df_POD_season, how='left', on=['Season', 'LeagueName'])

## Normalize POD by demeaning and standardizing
df_TeamPOD_season_merged['POD_norm'] = ((df_TeamPOD_season_merged['POD'] - df_TeamPOD_season_merged['POD mean'])/df_TeamPOD_season_merged['POD std'])

## Bin normalized POD
df_TeamPOD_season_merged['PODbin'] = pd.cut(df_TeamPOD_season_merged.POD_norm, 24, labels=False)
df_TeamPOD_season_merged.head()

,Season,LeagueName,TeamID,POD,POD mean,POD std,POD_norm,PODbin
0,2010,??,3147,126.655172,126.655172,18.792620,0.000000,9
1,2010,AAC,3153,136.448276,152.817175,18.424465,-0.888433,4
2,2010,AAC,3163,156.718750,152.817175,18.424465,0.211761,11
3,2010,AAC,3187,146.741935,152.817175,18.424465,-0.329738,7
4,2010,AAC,3222,162.645161,152.817175,18.424465,0.533420,13


In [17]:
## Isolate OCR by team regardless of who won a game

df_WTeamOCR_game = df[['Season','WLeagueName', 'WTeamID','WOCR']].rename(columns = {'WLeagueName':'LeagueName', 'WTeamID':'TeamID', 'WOCR':'OCR'})
df_LTeamOCR_game = df[['Season','LLeagueName', 'LTeamID','LOCR']].rename(columns = {'LLeagueName':'LeagueName', 'LTeamID':'TeamID', 'LOCR':'OCR'})
df_TeamOCR_game = pd.concat([df_WTeamOCR_game, df_LTeamOCR_game])

df_TeamOCR_season = df_TeamOCR_game.groupby(['Season', 'LeagueName', 'TeamID'], as_index=False).agg({'OCR':['mean']})
df_TeamOCR_season.columns = df_TeamOCR_season.columns.droplevel(1)

df_OCR_season = df_TeamOCR_game.groupby(['Season', 'LeagueName'], as_index=False).agg({'OCR':['mean','std']})

df_OCR_season.columns = [' '.join(col).strip() for col in df_OCR_season.columns.values] ## Flatten column name levels

df_TeamOCR_season_merged = pd.merge(left=df_TeamOCR_season, right=df_OCR_season, how='left', on=['Season', 'LeagueName'])

## Normalize OCR by demeaning and standardizing
df_TeamOCR_season_merged['OCR_norm'] = ((df_TeamOCR_season_merged['OCR'] - df_TeamOCR_season_merged['OCR mean'])/df_TeamOCR_season_merged['OCR std'])

## Bin normalized OCR
df_TeamOCR_season_merged['OCRbin'] = pd.cut(df_TeamOCR_season_merged.OCR_norm, 24, labels=False)
df_TeamOCR_season_merged.tail()

,Season,LeagueName,TeamID,OCR,OCR mean,OCR std,OCR_norm,OCRbin
3113,2018,WCC,3339,0.408454,0.43462,0.074073,-0.353252,7
3114,2018,WCC,3360,0.436588,0.43462,0.074073,0.026566,11
3115,2018,WCC,3362,0.429383,0.43462,0.074073,-0.070704,10
3116,2018,WCC,3365,0.374669,0.43462,0.074073,-0.809351,3
3117,2018,WCC,3388,0.494649,0.43462,0.074073,0.810400,18


In [18]:
data_dir = './WStage2DataFiles/'
df_seeds = pd.read_csv(data_dir + 'WNCAATourneySeeds.csv')
df_tour = pd.read_csv(data_dir + 'WNCAATourneyCompactResults.csv')

##df_tour = df_tour[df_tour['Season'] > 2012]
df_tour.shape


(1260, 8)

In [19]:
## Create a dataframe with regular season engineeried features
## An engineered feature here reflects the performance of a team during a season on a performance type dimension

df_temp = pd.merge(left=df_TeamFGP_season_merged, right=df_TeamAst_season_merged, how='left', on=['Season', 'TeamID'])
df_temp = pd.merge(left=df_temp, right=df_TeamBlk_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_temp, right=df_TeamFGA_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamFTP_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamFTA_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamOR_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamDR_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamStl_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamPOD_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_TeamOCR_season_merged, how='left', on=['Season', 'TeamID'])
df_engineered_features = pd.merge(left=df_engineered_features, right=df_team_league_perf, how='left', on=['TeamID'])

print df_engineered_features.shape
df_engineered_features.tail()



(3118, 69)


,Season,LeagueName_x,TeamID,FGP,FGP mean,FGP std,FGP_norm,FGPbin,LeagueName_y,Ast,...,POD std,POD_norm,PODbin,LeagueName,OCR,OCR mean,OCR std,OCR_norm,OCRbin,Leaguebin
3113,2018,WCC,3339,0.388120,0.412622,0.078129,-0.313611,7,WCC,13.071429,...,19.764475,-0.085895,9,WCC,0.408454,0.43462,0.074073,-0.353252,7,6.0
3114,2018,WCC,3360,0.411764,0.412622,0.078129,-0.010987,10,WCC,13.838710,...,19.764475,-0.365688,7,WCC,0.436588,0.43462,0.074073,0.026566,11,6.0
3115,2018,WCC,3362,0.415736,0.412622,0.078129,0.039848,10,WCC,15.419355,...,19.764475,0.341022,12,WCC,0.429383,0.43462,0.074073,-0.070704,10,6.0
3116,2018,WCC,3365,0.349105,0.412622,0.078129,-0.812980,3,WCC,11.482759,...,19.764475,-0.986401,3,WCC,0.374669,0.43462,0.074073,-0.809351,3,6.0
3117,2018,WCC,3388,0.483855,0.412622,0.078129,0.911732,18,WCC,15.966667,...,19.764475,-0.440548,7,WCC,0.494649,0.43462,0.074073,0.810400,18,6.0


In [20]:
df.shape

(46342, 44)

### Calculate a win probability for each pair of teams during regular season based on regular season data


In [21]:
### Recast regular season games a pairwise team cobinations within seasons
df_wins = pd.DataFrame()
df_wins['Season'] = df['Season']
df_wins['First_TeamID'] = df['WTeamID']
df_wins['Second_TeamID'] = df['LTeamID']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['Season'] = df['Season']
df_losses['First_TeamID'] = df['LTeamID']
df_losses['Second_TeamID'] = df['WTeamID']
df_losses['Result'] = 0

df_regular_season_games = pd.concat((df_wins, df_losses))
df_regular_season_games.tail()

,Season,First_TeamID,Second_TeamID,Result
46337,2018,3239,3195,0
46338,2018,3421,3251,0
46339,2018,3372,3311,0
46340,2018,3335,3343,0
46341,2018,3352,3384,0


In [22]:
df_regular_season_games.shape

(92684, 4)

### Inject engineered features

In [23]:
df_engineered_features = df_engineered_features[['Season', 'TeamID', 'FGPbin', 'FTPbin', 'FGAbin', 'FTAbin', 'Astbin', 'Blkbin', 'ORbin', 'DRbin', 'Leaguebin', 'Stlbin', 'PODbin', 'OCRbin']]

df_engineered_features_W = df_engineered_features.rename(columns={'Leaguebin':'WLeaguebin', 'TeamID':'WTeamID', 'FGPbin':'WFGPbin', 'FTPbin':'WFTPbin', 'FGAbin':'WFGAbin', 'FTAbin':'WFTAbin', 'Astbin':'WAstbin', 'Blkbin':'WBlkbin', 'ORbin':'WORbin', 'DRbin':'WDRbin', 'Stlbin':'WStlbin', 'PODbin':'WPODbin', 'OCRbin':'WOCRbin'})
df_engineered_features_L = df_engineered_features.rename(columns={'Leaguebin':'LLeaguebin', 'TeamID':'LTeamID', 'FGPbin':'LFGPbin', 'FTPbin':'LFTPbin', 'FGAbin':'LFGAbin', 'FTAbin':'LFTAbin', 'Astbin':'LAstbin', 'Blkbin':'LBlkbin', 'ORbin':'LORbin', 'DRbin':'LDRbin', 'Stlbin':'LStlbin', 'PODbin':'LPODbin', 'OCRbin':'LOCRbin'})

df_first_team_features = df_engineered_features_W
df_first_team_features = df_first_team_features.rename(columns = {'WTeamID':'First_TeamID', 'LTeamID':'Second_TeamID', 'WFGPbin':'First_FGPbin', 'WPODbin':'First_PODbin', 'WOCRbin':'First_OCRbin', 'WFTPbin':'First_FTPbin', 'WFGAbin':'First_FGAbin', 'WFTAbin':'First_FTAbin', 'WAstbin':'First_Astbin', 'WBlkbin':'First_Blkbin', 'WORbin':'First_ORbin', 'WDRbin':'First_DRbin', 'WStlbin':'First_Stlbin', 'WLeaguebin':'First_Leaguebin', 'WSeed':'First_Seed', 'LFGPbin':'Second_FGPbin', 'LFTPbin':'Second_FTPbin', 'LPODbin':'Second_PODbin', 'LOCRbin':'Second_OCRbin', 'LAstbin':'Second_Astbin', 'LBlkbin':'Second_Blkbin', 'LORbin':'Second_ORbin', 'LDRbin':'Second_DRbin', 'LStlbin':'Second_Stlbin', 'LLeaguebin':'Second_Leaguebin', 'LSeed':'Second_Seed'})

df_regular_season_games_with_features = pd.merge(left=df_regular_season_games, right=df_first_team_features, on=['Season', 'First_TeamID'])

df_second_team_features = df_engineered_features_L
df_second_team_features = df_second_team_features.rename(columns = {'WTeamID':'First_TeamID', 'LTeamID':'Second_TeamID', 'WFGPbin':'First_FGPbin', 'WFTPbin':'First_FTPbin', 'WFGAbin':'First_FGAbin', 'WPODbin':'First_PODbin', 'WOCRbin':'First_OCRbin', 'WFTAbin':'First_FTAbin', 'WAstbin':'First_Astbin', 'WBlkbin':'First_Blkbin', 'WORbin':'First_ORbin', 'WDRbin':'First_DRbin', 'WStlbin':'First_Stlbin', 'WLeaguebin':'First_Leaguebin', 'WSeed':'First_Seed', 'LFGPbin':'Second_FGPbin', 'LFTPbin':'Second_FTPbin', 'LPODbin':'Second_PODbin', 'LOCRbin':'Second_OCRbin', 'LAstbin':'Second_Astbin', 'LBlkbin':'Second_Blkbin', 'LORbin':'Second_ORbin', 'LDRbin':'Second_DRbin', 'LStlbin':'Second_Stlbin', 'LLeaguebin':'Second_Leaguebin', 'LSeed':'Second_Seed'})

df_regular_season_games_with_features = pd.merge(left=df_regular_season_games_with_features, right=df_second_team_features, on=['Season', 'Second_TeamID'])
df_regular_season_games_with_features.head()

,Season,First_TeamID,Second_TeamID,Result,First_FGPbin,First_FTPbin,First_FGAbin,First_FTAbin,First_Astbin,First_Blkbin,...,LFGAbin,LFTAbin,Second_Astbin,Second_Blkbin,Second_ORbin,Second_DRbin,Second_Leaguebin,Second_Stlbin,Second_PODbin,Second_OCRbin
0,2010,3103,3237,1,10,14,7,12,10,6,...,5,14,2,15,12,11,4.0,7,5,6
1,2010,3231,3237,1,6,12,14,14,6,2,...,5,14,2,15,12,11,4.0,7,5,6
2,2010,3282,3237,1,12,14,10,11,10,7,...,5,14,2,15,12,11,4.0,7,5,6
3,2010,3282,3237,1,12,14,10,11,10,7,...,5,14,2,15,12,11,4.0,7,5,6
4,2010,3293,3237,1,12,19,13,12,12,13,...,5,14,2,15,12,11,4.0,7,5,6


In [24]:
df_temp.tail()

,Season,LeagueName_x,TeamID,FGP,FGP mean,FGP std,FGP_norm,FGPbin,LeagueName_y,Ast,Ast mean,Ast std,Ast_norm,Astbin,LeagueName,Blk,Blk mean,Blk std,Blk_norm,Blkbin
3113,2018,WCC,3339,0.388120,0.412622,0.078129,-0.313611,7,WCC,13.071429,14.149502,4.394039,-0.245349,7,WCC,1.964286,3.196013,2.388469,-0.515698,5
3114,2018,WCC,3360,0.411764,0.412622,0.078129,-0.010987,10,WCC,13.838710,14.149502,4.394039,-0.070730,9,WCC,1.387097,3.196013,2.388469,-0.757354,3
3115,2018,WCC,3362,0.415736,0.412622,0.078129,0.039848,10,WCC,15.419355,14.149502,4.394039,0.288995,11,WCC,2.225806,3.196013,2.388469,-0.406204,6
3116,2018,WCC,3365,0.349105,0.412622,0.078129,-0.812980,3,WCC,11.482759,14.149502,4.394039,-0.606900,5,WCC,3.275862,3.196013,2.388469,0.033431,9
3117,2018,WCC,3388,0.483855,0.412622,0.078129,0.911732,18,WCC,15.966667,14.149502,4.394039,0.413552,12,WCC,3.233333,3.196013,2.388469,0.015625,9


### Train model on regular season data to predict wins based on regular season performance

In [25]:
df_temp = df_regular_season_games_with_features.dropna(how='any')
df_temp = df_temp[['Result', 'Season', 'First_TeamID', 'Second_TeamID', 'First_PODbin', 'Second_PODbin', 'First_OCRbin', 'Second_OCRbin', 'First_Leaguebin', 'Second_Leaguebin', 'First_Astbin', 'Second_Astbin', 'First_Blkbin', 'Second_Blkbin', 'First_ORbin', 'Second_ORbin', 'First_Stlbin', 'Second_Stlbin']]

##cat1 = pd.get_dummies(df_temp['First_TeamID'], prefix = 'First_TeamID')
##cat2 = pd.get_dummies(df_temp['Second_TeamID'], prefix = 'Second_TeamID')
##cat3 = pd.get_dummies(df_temp['Season'], prefix = 'Season')

##df_temp = pd.concat([df_temp, cat1, cat2],axis=1)
##df_temp = df_temp.drop('First_TeamID', 1)
##df_temp = df_temp.drop('Second_TeamID', 1)

##df_temp = df_temp.drop({ORbinDiff','DRbinDiff', 'FGAbinDiff', 'FGPbinDiff', 'FTAbinDiff', 'AstbinDiff', 'StlbinDiff', 'FTPbinDiff'}, 1)

################
X_train = df_temp[df_temp['Season'] < 2018]
X_test = df_temp[df_temp['Season'] == 2018]
################


##X_train = X_train.drop({'SeedDiff'}, 1)
##X_test = X_test.drop({'SeedDiff'}, 1)


## cat5 = pd.get_dummies(X_train['Season'], prefix = 'Season')
## X_train = pd.concat([X_train, cat5],axis=1)

##X_train = X_train.drop('Season', 1)
##X_test = X_test.drop('Season', 1)


##X_train = X_train.drop('Season', 1)
##X_test = X_test.drop('Season', 1)

y_train = X_train.Result.values
y_test = X_test.Result.values

X_train = X_train.drop('Result', 1)
X_test = X_test.drop('Result', 1)

##X_train['RbinDiff'] = X_train.ORbinDiff + X_train.DRbinDiff

## X_temp = df_temp.drop({'ORbinDiff','DRbinDiff', 'AstbinDiff', 'FGPbinDiff', 'FGAbinDiff', 'FTPbinDiff', 'FTAbinDiff', 'LeaguebinDiff'}, 1)
## X_temp = df_temp[['SeedDiff', 'FGPbinDiff', 'AstbinDiff', 'BlkbinDiff']]


X_train.tail()


,Season,First_TeamID,Second_TeamID,First_PODbin,Second_PODbin,First_OCRbin,Second_OCRbin,First_Leaguebin,Second_Leaguebin,First_Astbin,Second_Astbin,First_Blkbin,Second_Blkbin,First_ORbin,Second_ORbin,First_Stlbin,Second_Stlbin
82261,2017,3223,3146,7,4,4,18,1.0,1.0,3,13,12,11,9,10,1,2
82262,2017,3251,3146,12,4,10,18,0.0,1.0,10,13,16,11,20,10,6,2
82263,2017,3108,3146,12,4,6,18,0.0,1.0,9,13,12,11,14,10,10,2
82264,2017,3108,3146,12,4,6,18,0.0,1.0,9,13,12,11,14,10,10,2
82265,2017,3358,3146,5,4,5,18,1.0,1.0,6,13,12,11,10,10,3,2


In [26]:
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

def eval(clf):
    clf.fit(X_train,y_train)
    print "score = " + str(clf.score(X_test, y_test))

    pred = clf.predict_proba(X_test)
    print "log loss = " + str(metrics.log_loss(y_test,pred))

print "fitting logistic regression model"
modelLR = LogisticRegression()
eval(modelLR)

print "\nfitting Naive Bayes model"
modelNB = GaussianNB()
eval(modelNB)

print "\nfitting Random Forest model"
modelRF = RandomForestClassifier(n_jobs=2, random_state=0)
eval(modelRF)

fitting logistic regression model
score = 0.736417738529
log loss = 0.517229243381

fitting Naive Bayes model
score = 0.717796122096
log loss = 0.579345348845

fitting Random Forest model
score = 0.700998272221
log loss = 0.692909645744


### Engineer a feature 'winprob' for tournament matchups based on regular season data

In [27]:
df_tour = df_tour[['Season', 'WTeamID', 'LTeamID']]
df_tour.head()

,Season,WTeamID,LTeamID
0,1998,3104,3422
1,1998,3112,3365
2,1998,3163,3193
3,1998,3198,3266
4,1998,3203,3208


In [28]:
df_Tour_2018 = pd.read_csv('NCAA_Result_2018_sep.csv')
df_Tour_2018 = df_Tour_2018.rename(columns={'Team1':'WTeamID', 'Team2':'LTeamID'})
df_Tour_2018 = df_Tour_2018.drop('Result', 1)
##df_tour_matchups = pd.concat([df_tour_matchups, df_Tour_2018], ignore_index=False)
df_Tour_2018.head()
df_concat = pd.concat([df_tour, df_Tour_2018], ignore_index=False)
df_concat.tail()

,Season,WTeamID,LTeamID
58,2018,3333,3443
59,2018,3333,3397
60,2018,3355,3437
61,2018,3376,3438
62,2018,3400,3417


In [29]:
df_concat.shape

(1323, 3)

In [30]:
## Inject regular season engineered feature into touranement data for the corresponding seasons and teams

df_engineered_features = df_engineered_features[['Season', 'TeamID', 'FGPbin', 'FTPbin', 'FGAbin', 'FTAbin', 'Astbin', 'Blkbin', 'ORbin', 'DRbin', 'Leaguebin', 'Stlbin', 'PODbin', 'OCRbin']]

# merge seeds with team IDs for tourney performance
df_W = df_seeds.rename(columns={'TeamID':'WTeamID', 'Seed':'WSeed'})
df_L = df_seeds.rename(columns={'TeamID':'LTeamID', 'Seed':'LSeed'})

##df_engineered_features_W = df_engineered_features.rename(columns={'Leaguebin':'WLeaguebin', 'TeamID':'WTeamID', 'FGPbin':'WFGPbin', 'FTPbin':'WFTPbin', 'FGAbin':'WFGAbin', 'FTAbin':'WFTAbin', 'Astbin':'WAstbin', 'Blkbin':'WBlkbin', 'ORbin':'WORbin', 'DRbin':'WDRbin', 'Stlbin':'WStlbin', 'PODbin':'WPODbin', 'OCRbin':'WOCRbin'})
##df_engineered_features_L = df_engineered_features.rename(columns={'Leaguebin':'LLeaguebin', 'TeamID':'LTeamID', 'FGPbin':'LFGPbin', 'FTPbin':'LFTPbin', 'FGAbin':'LFGAbin', 'FTAbin':'LFTAbin', 'Astbin':'LAstbin', 'Blkbin':'LBlkbin', 'ORbin':'LORbin', 'DRbin':'LDRbin', 'Stlbin':'LStlbin', 'PODbin':'LPODbin', 'OCRbin':'LOCRbin'})


df_concat = pd.merge(left=df_concat, right=df_W, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_L, on=['Season', 'LTeamID']) ## 

df_concat = pd.merge(left=df_concat, right=df_engineered_features_W, on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_engineered_features_L, how='left', on=['Season', 'LTeamID'])


# at the beginning of the tourney, teams play within their region
# final 3 games = between regions

print df_concat.shape
df_concat.tail()



(567, 29)


,Season,WTeamID,LTeamID,WSeed,LSeed,WFGPbin,WFTPbin,WFGAbin,WFTAbin,WAstbin,...,LFGAbin,LFTAbin,LAstbin,LBlkbin,LORbin,LDRbin,LLeaguebin,LStlbin,LPODbin,LOCRbin
562,2018,3333,3443,Z06,Z11,16,13,8,6,15,...,16,17,9,13,19,6,12.0,15,14,14
563,2018,3276,3294,Z07,Z10,14,16,8,12,9,...,6,11,12,9,5,7,0.0,11,7,15
564,2018,3355,3437,X08,X09,13,18,12,15,14,...,8,7,9,13,7,15,15.0,2,9,10
565,2018,3329,3393,Y09,Y08,9,17,13,18,9,...,13,9,12,7,13,10,22.0,9,15,8
566,2018,3332,3370,X02,X15,17,17,10,14,15,...,15,13,12,9,13,21,0.0,7,13,12


In [31]:
df_tour_matchups = df_concat[['Season', 'WTeamID', 'LTeamID']]
df_tour_matchups.head()

,Season,WTeamID,LTeamID
0,2010,3124,3201
1,2010,3124,3207
2,2010,3124,3397
3,2010,3124,3181
4,2010,3173,3395


In [32]:
df_wins = pd.DataFrame()
df_wins['Season'] = df_tour_matchups['Season']
df_wins['First_TeamID'] = df_tour_matchups['WTeamID']
df_wins['Second_TeamID'] = df_tour_matchups['LTeamID']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['Season'] = df_tour_matchups['Season']
df_losses['First_TeamID'] = df_tour_matchups['LTeamID']
df_losses['Second_TeamID'] = df_tour_matchups['WTeamID']
df_losses['Result'] = 0

df_tour_matchups = pd.concat((df_wins, df_losses))
df_tour_matchups.head()

,Season,First_TeamID,Second_TeamID,Result
0,2010,3124,3201,1
1,2010,3124,3207,1
2,2010,3124,3397,1
3,2010,3124,3181,1
4,2010,3173,3395,1


In [33]:
### Inject engineered features into Tournament matchups
df_first_team_features = df_engineered_features_W
df_first_team_features = df_first_team_features.rename(columns = {'WTeamID':'First_TeamID', 'LTeamID':'Second_TeamID', 'WFGPbin':'First_FGPbin', 'WPODbin':'First_PODbin', 'WOCRbin':'First_OCRbin', 'WFTPbin':'First_FTPbin', 'WFGAbin':'First_FGAbin', 'WFTAbin':'First_FTAbin', 'WAstbin':'First_Astbin', 'WBlkbin':'First_Blkbin', 'WORbin':'First_ORbin', 'WDRbin':'First_DRbin', 'WStlbin':'First_Stlbin', 'WLeaguebin':'First_Leaguebin', 'WSeed':'First_Seed', 'LFGPbin':'Second_FGPbin', 'LFTPbin':'Second_FTPbin', 'LPODbin':'Second_PODbin', 'LOCRbin':'Second_OCRbin', 'LAstbin':'Second_Astbin', 'LBlkbin':'Second_Blkbin', 'LORbin':'Second_ORbin', 'LDRbin':'Second_DRbin', 'LStlbin':'Second_Stlbin', 'LLeaguebin':'Second_Leaguebin', 'LSeed':'Second_Seed'})

df_tour_matchups_with_features = pd.merge(left=df_tour_matchups, right=df_first_team_features, on=['Season', 'First_TeamID'])

df_second_team_features = df_engineered_features_L
df_second_team_features = df_second_team_features.rename(columns = {'WTeamID':'First_TeamID', 'LTeamID':'Second_TeamID', 'WFGPbin':'First_FGPbin', 'WFTPbin':'First_FTPbin', 'WFGAbin':'First_FGAbin', 'WPODbin':'First_PODbin', 'WOCRbin':'First_OCRbin', 'WFTAbin':'First_FTAbin', 'WAstbin':'First_Astbin', 'WBlkbin':'First_Blkbin', 'WORbin':'First_ORbin', 'WDRbin':'First_DRbin', 'WStlbin':'First_Stlbin', 'WLeaguebin':'First_Leaguebin', 'WSeed':'First_Seed', 'LFGPbin':'Second_FGPbin', 'LFTPbin':'Second_FTPbin', 'LPODbin':'Second_PODbin', 'LOCRbin':'Second_OCRbin', 'LAstbin':'Second_Astbin', 'LBlkbin':'Second_Blkbin', 'LORbin':'Second_ORbin', 'LDRbin':'Second_DRbin', 'LStlbin':'Second_Stlbin', 'LLeaguebin':'Second_Leaguebin', 'LSeed':'Second_Seed'})

df_tour_matchups_with_features = pd.merge(left=df_tour_matchups_with_features, right=df_second_team_features, on=['Season', 'Second_TeamID'])
df_tour_matchups_with_features.tail()

,Season,First_TeamID,Second_TeamID,Result,First_FGPbin,First_FTPbin,First_FGAbin,First_FTAbin,First_Astbin,First_Blkbin,...,LFGAbin,LFTAbin,Second_Astbin,Second_Blkbin,Second_ORbin,Second_DRbin,Second_Leaguebin,Second_Stlbin,Second_PODbin,Second_OCRbin
1129,2018,3328,3177,0,11,12,10,17,6,10,...,17,12,18,8,15,15,15.0,10,18,12
1130,2018,3401,3179,0,13,19,11,17,9,8,...,13,14,22,10,12,15,4.0,12,14,21
1131,2018,3437,3355,0,11,15,8,7,9,13,...,12,15,14,13,14,17,1.0,7,12,15
1132,2018,3438,3143,0,8,15,7,7,7,10,...,8,13,10,7,11,12,21.0,4,7,13
1133,2018,3397,3251,0,12,14,12,20,12,11,...,9,13,13,11,16,10,0.0,10,8,15


In [34]:
## Use Naive Bayes probs
df_tour_matchups = df_tour_matchups_with_features[['Result', 'Season', 'First_TeamID', 'Second_TeamID', 'First_PODbin', 'Second_PODbin', 'First_OCRbin', 'Second_OCRbin', 'First_Astbin', 'Second_Astbin', 'First_Blkbin', 'Second_Blkbin', 'First_ORbin', 'Second_ORbin', 'First_Stlbin', 'Second_Stlbin', 'First_Leaguebin', 'Second_Leaguebin']]
df_tour_matchups = df_tour_matchups.drop('Result', 1)
df_temp = pd.DataFrame(modelNB.predict_proba(df_tour_matchups), columns=list('xy'))
df_tour_matchups.head()

,Season,First_TeamID,Second_TeamID,First_PODbin,Second_PODbin,First_OCRbin,Second_OCRbin,First_Astbin,Second_Astbin,First_Blkbin,Second_Blkbin,First_ORbin,Second_ORbin,First_Stlbin,Second_Stlbin,First_Leaguebin,Second_Leaguebin
0,2010,3124,3201,7,15,16,13,11,12,18,6,9,12,7,12,21.0,7.0
1,2010,3124,3207,7,13,16,10,11,13,18,6,9,16,7,17,21.0,15.0
2,2010,3265,3207,11,13,17,10,13,13,13,6,7,16,8,17,3.0,15.0
3,2010,3124,3397,7,9,16,16,11,12,18,17,9,13,7,5,21.0,23.0
4,2010,3173,3397,15,9,12,16,13,12,14,17,16,13,7,5,10.0,23.0


In [35]:
df_tour_matchups_with_features['winprob'] = df_temp.x
df_tour_matchups_with_features['lossprob'] = df_temp.y

df_tour_matchups_with_features = df_tour_matchups_with_features[['Season', 'First_TeamID', 'Second_TeamID', 'winprob', 'lossprob']]
df_tour_matchups_with_features.head()

,Season,First_TeamID,Second_TeamID,winprob,lossprob
0,2010,3124,3201,0.013256,0.986744
1,2010,3124,3207,0.030878,0.969122
2,2010,3265,3207,0.401893,0.598107
3,2010,3124,3397,0.623502,0.376498
4,2010,3173,3397,0.958442,0.041558


In [36]:
df_tour_matchups_with_features.tail()

,Season,First_TeamID,Second_TeamID,winprob,lossprob
1129,2018,3328,3177,0.530272,0.469728
1130,2018,3401,3179,0.664851,0.335149
1131,2018,3437,3355,0.737012,0.262988
1132,2018,3438,3143,0.607101,0.392899
1133,2018,3397,3251,0.095365,0.904635


### Calculate a win probability for each pair of teams during a tournament season based on regular season data


In [37]:
# calculate difference in performance based for all features  
df_concat['SeedDiff'] = df_concat.apply(lambda row: int(row['WSeed'][1:3]) - int(row['LSeed'][1:3]), axis=1)
df_concat['SeedDiffPct'] = df_concat['SeedDiff'] / (df_concat.apply(lambda row: int(row['WSeed'][1:3]) + int(row['LSeed'][1:3]), axis=1))
df_concat['FGPbinDiff'] = df_concat.apply(lambda row: int(row['WFGPbin']) - int(row['LFGPbin']), axis=1)
df_concat['FTPbinDiff'] = df_concat.apply(lambda row: int(row['WFTPbin']) - int(row['LFTPbin']), axis=1)
df_concat['FGAbinDiff'] = df_concat.apply(lambda row: int(row['WFGAbin']) - int(row['LFGAbin']), axis=1)
df_concat['FTAbinDiff'] = df_concat.apply(lambda row: int(row['WFTAbin']) - int(row['LFTAbin']), axis=1)
df_concat['AstbinDiff'] = df_concat.apply(lambda row: int(row['WAstbin']) - int(row['LAstbin']), axis=1)
df_concat['BlkbinDiff'] = df_concat.apply(lambda row: int(row['WBlkbin']) - int(row['LBlkbin']), axis=1)
df_concat['ORbinDiff'] = df_concat.apply(lambda row: int(row['WORbin']) - int(row['LORbin']), axis=1)
df_concat['DRbinDiff'] = df_concat.apply(lambda row: int(row['WDRbin']) - int(row['LDRbin']), axis=1)
df_concat['StlbinDiff'] = df_concat.apply(lambda row: int(row['WStlbin']) - int(row['LStlbin']), axis=1)
df_concat['PODbinDiff'] = df_concat.apply(lambda row: int(row['WPODbin']) - int(row['LPODbin']), axis=1)
df_concat['OCRbinDiff'] = df_concat.apply(lambda row: int(row['WOCRbin']) - int(row['LOCRbin']), axis=1)
df_concat['LeaguebinDiff'] = df_concat.apply(lambda row: int(row['WLeaguebin']) - int(row['LLeaguebin']), axis=1)

df_concat.tail(10)

,Season,WTeamID,LTeamID,WSeed,LSeed,WFGPbin,WFTPbin,WFGAbin,WFTAbin,WAstbin,...,FGAbinDiff,FTAbinDiff,AstbinDiff,BlkbinDiff,ORbinDiff,DRbinDiff,StlbinDiff,PODbinDiff,OCRbinDiff,LeaguebinDiff
557,2018,3195,3390,Z12,Z04,15,13,13,10,16,...,2,-1,9,-9,-2,-2,4,5,7,-19
558,2018,3211,3390,Z13,Z04,13,16,11,14,10,...,0,3,3,-4,2,-3,5,0,6,-15
559,2018,3208,3273,W04,W13,9,11,11,8,12,...,-4,0,-5,5,-1,6,-4,-3,-8,23
560,2018,3251,3397,Z14,Z03,15,15,9,13,13,...,-3,-7,1,0,2,-8,0,-4,2,-23
561,2018,3333,3397,Z06,Z03,16,13,8,6,15,...,-4,-14,3,6,-4,3,-10,-5,3,-2
562,2018,3333,3443,Z06,Z11,16,13,8,6,15,...,-8,-11,6,4,-9,15,-15,-7,2,9
563,2018,3276,3294,Z07,Z10,14,16,8,12,9,...,2,1,-3,-2,6,7,-5,1,1,17
564,2018,3355,3437,X08,X09,13,18,12,15,14,...,4,8,5,0,7,2,5,3,5,-14
565,2018,3329,3393,Y09,Y08,9,17,13,18,9,...,0,9,-3,-3,-1,1,1,-1,2,-1
566,2018,3332,3370,X02,X15,17,17,10,14,15,...,-5,1,3,1,-3,-8,1,-2,7,21


In [38]:
df_wins = pd.DataFrame()

df_wins['SeedDiff'] = df_concat['SeedDiff']
df_wins['SeedDiffPct'] = df_concat['SeedDiffPct']
df_wins['FGPbinDiff'] = df_concat['FGPbinDiff']
df_wins['FTPbinDiff'] = df_concat['FTPbinDiff']
df_wins['FGAbinDiff'] = df_concat['FGAbinDiff']
df_wins['FTAbinDiff'] = df_concat['FTAbinDiff']
df_wins['AstbinDiff'] = df_concat['AstbinDiff']
df_wins['BlkbinDiff'] = df_concat['BlkbinDiff']
df_wins['ORbinDiff'] = df_concat['ORbinDiff']
df_wins['DRbinDiff'] = df_concat['DRbinDiff']
df_wins['StlbinDiff'] = df_concat['StlbinDiff']
df_wins['PODbinDiff'] = df_concat['PODbinDiff']
df_wins['OCRbinDiff'] = df_concat['OCRbinDiff']
df_wins['LeaguebinDiff'] = df_concat['LeaguebinDiff']
df_wins['Season'] = df_concat['Season']
df_wins['First_TeamID'] = df_concat['WTeamID']
df_wins['Second_TeamID'] = df_concat['LTeamID']


## df_wins['DayNum'] = df_concat['DayNum']



df_wins['Result'] = 1

df_losses = pd.DataFrame()

df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['SeedDiffPct'] = -df_concat['SeedDiffPct']
df_losses['FGPbinDiff'] = -df_concat['FGPbinDiff']
df_losses['FTPbinDiff'] = -df_concat['FTPbinDiff']
df_losses['FGAbinDiff'] = -df_concat['FGAbinDiff']
df_losses['FTAbinDiff'] = -df_concat['FTAbinDiff']
df_losses['AstbinDiff'] = -df_concat['AstbinDiff']
df_losses['BlkbinDiff'] = -df_concat['BlkbinDiff']
df_losses['ORbinDiff'] = -df_concat['ORbinDiff']
df_losses['DRbinDiff'] = -df_concat['DRbinDiff']
df_losses['StlbinDiff'] = -df_concat['StlbinDiff']
df_losses['PODbinDiff'] = -df_concat['PODbinDiff']
df_losses['OCRbinDiff'] = -df_concat['OCRbinDiff']
df_losses['LeaguebinDiff'] = -df_concat['LeaguebinDiff']
df_losses['Season'] = df_concat['Season']
df_losses['First_TeamID'] = df_concat['LTeamID']
df_losses['Second_TeamID'] = df_concat['WTeamID']
## df_losses['DayNum'] = df_concat['DayNum']



df_losses['Result'] = 0

df_predictions = pd.concat((df_wins, df_losses))
df_predictions.head()


,SeedDiff,SeedDiffPct,FGPbinDiff,FTPbinDiff,FGAbinDiff,FTAbinDiff,AstbinDiff,BlkbinDiff,ORbinDiff,DRbinDiff,StlbinDiff,PODbinDiff,OCRbinDiff,LeaguebinDiff,Season,First_TeamID,Second_TeamID,Result
0,-9,-0.529412,2,2,-6,6,-1,12,-3,8,-5,-8,3,14,2010,3124,3201,1
1,-1,-0.111111,5,0,-4,4,-2,12,-7,19,-10,-6,6,6,2010,3124,3207,1
2,3,0.600000,-1,3,-3,9,-1,1,-4,3,2,-2,0,-2,2010,3124,3397,1
3,2,0.333333,3,5,-4,8,1,4,-5,9,-7,-4,5,-1,2010,3124,3181,1
4,-1,-0.058824,4,-4,2,5,0,6,4,9,-5,2,2,-11,2010,3173,3395,1


In [39]:
df_temp = df_predictions[['First_TeamID', 'Second_TeamID', 'SeedDiffPct', 'Season', 'Result', 'LeaguebinDiff', 'PODbinDiff', 'OCRbinDiff']]
df_temp = pd.merge(left=df_temp, right=df_tour_matchups_with_features, how='left', on=['Season', 'First_TeamID', 'Second_TeamID'])


In [40]:
df_temp.tail()

,First_TeamID,Second_TeamID,SeedDiffPct,Season,Result,LeaguebinDiff,PODbinDiff,OCRbinDiff,winprob,lossprob
1129,3443,3333,0.294118,2018,0,-9,7,-2,0.836701,0.163299
1130,3294,3276,0.176471,2018,0,-17,-1,-1,0.802445,0.197555
1131,3437,3355,0.058824,2018,0,14,-3,-5,0.737012,0.262988
1132,3393,3329,-0.058824,2018,0,1,1,-2,0.405514,0.594486
1133,3370,3332,0.764706,2018,0,-21,2,-7,0.980472,0.019528


In [41]:
# Convert to arrays
## X_train = df_predictions.SeedDiff.values.reshape(-1,1)
## y_train = df_predictions.Result.values

df_temp = df_predictions[['First_TeamID', 'Second_TeamID', 'SeedDiffPct', 'Season', 'Result', 'LeaguebinDiff', 'PODbinDiff', 'OCRbinDiff']]
df_temp = pd.merge(left=df_temp, right=df_tour_matchups_with_features, how='left', on=['Season', 'First_TeamID', 'Second_TeamID'])
##df_temp = df_temp.drop({ 'lossprob' , 'First_TeamID', 'Second_TeamID',}, 1)
##df_temp['stage2pred'] = stage2pred


##df_temp = df_temp.drop({ORbinDiff','DRbinDiff', 'FGAbinDiff', 'FGPbinDiff', 'FTAbinDiff', 'AstbinDiff', 'StlbinDiff', 'FTPbinDiff'}, 1)

X_train = df_temp[df_temp['Season'] < 2018]
X_test = df_temp[df_temp['Season'] == 2018]

##X_train = X_train.drop({'SeedDiff'}, 1)
##X_test = X_test.drop({'SeedDiff'}, 1)


## cat5 = pd.get_dummies(X_train['Season'], prefix = 'Season')
## X_train = pd.concat([X_train, cat5],axis=1)


In [42]:
test_results = pd.read_csv('NCAA_Result_2018_sep.csv')
test_results= test_results.rename(columns={'Team1':'First_TeamID', 'Team2':'Second_TeamID'})
test_results.head()

,Season,First_TeamID,Second_TeamID,Result
0,2018,3110,3417,0
1,2018,3113,3304,1
2,2018,3113,3400,0
3,2018,3114,3199,0
4,2018,3124,3212,1


In [43]:
df_wins = pd.DataFrame()


df_wins['Season'] = test_results['Season']
df_wins['First_TeamID'] = test_results['First_TeamID']
df_wins['Second_TeamID'] = test_results['Second_TeamID']
df_wins['Result'] = test_results['Result']

df_losses = pd.DataFrame()


df_losses['Season'] = test_results['Season']
df_losses['First_TeamID'] = test_results['Second_TeamID']
df_losses['Second_TeamID'] = test_results['First_TeamID']
df_losses['Result'] = (1 - test_results['Result'])

test_result_2way = pd.concat((df_wins, df_losses))
test_result_2way.head()

,Season,First_TeamID,Second_TeamID,Result
0,2018,3110,3417,0
1,2018,3113,3304,1
2,2018,3113,3400,0
3,2018,3114,3199,0
4,2018,3124,3212,1


In [44]:
X_test = X_test.drop("Result", 1)
X_test = pd.merge(left=X_test, right=test_result_2way, how='left', on=['Season', 'First_TeamID', 'Second_TeamID'])
X_test.head()


,First_TeamID,Second_TeamID,SeedDiffPct,Season,LeaguebinDiff,PODbinDiff,OCRbinDiff,winprob,lossprob,Result
0,3110,3417,0.647059,2018,-21,-3,3,0.962222,0.037778,0
1,3166,3417,0.571429,2018,-6,-4,2,0.861062,0.138938,0
2,3166,3234,0.294118,2018,-2,0,-6,0.897916,0.102084,1
3,3280,3417,-0.500000,2018,2,0,6,0.307419,0.692581,1
4,3280,3323,0.000000,2018,1,1,-5,0.766554,0.233446,0


In [45]:
X_train.head()

,First_TeamID,Second_TeamID,SeedDiffPct,Season,Result,LeaguebinDiff,PODbinDiff,OCRbinDiff,winprob,lossprob
0,3124,3201,-0.529412,2010,1,14,-8,3,0.013256,0.986744
1,3124,3207,-0.111111,2010,1,6,-6,6,0.030878,0.969122
2,3124,3397,0.600000,2010,1,-2,-2,0,0.623502,0.376498
3,3124,3181,0.333333,2010,1,-1,-4,5,0.238812,0.761188
4,3173,3395,-0.058824,2010,1,-11,2,2,0.457081,0.542919


In [46]:

X_train = X_train.drop({'Season', 'First_TeamID', 'Second_TeamID', 'lossprob', 'PODbinDiff','OCRbinDiff'}, 1)
X_test = X_test.drop({'Season', 'First_TeamID', 'Second_TeamID', 'lossprob', 'PODbinDiff', 'OCRbinDiff'}, 1)


y_train = X_train.Result.values
y_test = X_test.Result.values

X_train = X_train.drop('Result', 1)
X_test = X_test.drop('Result', 1)

##X_train['RbinDiff'] = X_train.ORbinDiff + X_train.DRbinDiff

## X_temp = df_temp.drop({'ORbinDiff','DRbinDiff', 'AstbinDiff', 'FGPbinDiff', 'FGAbinDiff', 'FTPbinDiff', 'FTAbinDiff', 'LeaguebinDiff'}, 1)
## X_temp = df_temp[['SeedDiff', 'FGPbinDiff', 'AstbinDiff', 'BlkbinDiff']]


print X_train.head()
print X_test.head()


   SeedDiffPct  LeaguebinDiff   winprob
0    -0.529412             14  0.013256
1    -0.111111              6  0.030878
2     0.600000             -2  0.623502
3     0.333333             -1  0.238812
4    -0.058824            -11  0.457081
   SeedDiffPct  LeaguebinDiff   winprob
0     0.647059            -21  0.962222
1     0.571429             -6  0.861062
2     0.294118             -2  0.897916
3    -0.500000              2  0.307419
4     0.000000              1  0.766554


In [47]:
## Fit to training tournament matchups and predict test matchups

from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


def eval(clf):
    clf.fit(X_train,y_train)
    print "score = " + str(clf.score(X_test, y_test))

    pred = clf.predict_proba(X_test)
    print "log loss = " + str(metrics.log_loss(y_test,pred))

print "fitting logistic regression model"
model = LogisticRegression()
eval(model)

print "\nfitting SVM  model"
model = svm.SVC(probability=True)
eval(model)

print "\nfitting Naive Bayes model"
model = GaussianNB()
eval(model)

print "\nfitting Random Forest model"
model = RandomForestClassifier(n_jobs=2, random_state=0)
eval(model)

print "\nfitting Gradient Boosting Classifier"
model = GradientBoostingClassifier(n_estimators=10, learning_rate=1.0, max_depth=2, random_state=0)
eval(model)


fitting logistic regression model
score = 0.777777777778
log loss = 0.428648064655

fitting SVM  model
score = 0.777777777778
log loss = 0.478941300718

fitting Naive Bayes model
score = 0.777777777778
log loss = 0.560105767475

fitting Random Forest model
score = 0.722222222222
log loss = 1.8503717952

fitting Gradient Boosting Classifier
score = 0.81746031746
log loss = 0.449867825703
